In [5]:

import pandas as pd
import numpy as np
import re
from pathlib import Path

# Rutas a CSV 
files = {
    "altaverapaz": "datos_csv/altaverapaz.csv",
    "bajaverapaz": "datos_csv/bajaverapaz.csv",
    "chimaltenango": "datos_csv/chimaltenango.csv",
    "chiquimula": "datos_csv/chiquimula.csv",
    "ciudadcapital": "datos_csv/ciudadcapital.csv",
    "elprogreso": "datos_csv/elprogreso.csv",
    "escuintla": "datos_csv/escuintla.csv",
    "guatemala": "datos_csv/guatemala.csv",
    "izabal": "datos_csv/izabal.csv",
    "jalapa": "datos_csv/jalapa.csv",
    "jutiapa": "datos_csv/jutiapa.csv",
    "peten": "datos_csv/peten.csv",
    "quetzaltenango": "datos_csv/quetzaltenango.csv",
    "quiche": "datos_csv/quiche.csv",
    "retalhuleu": "datos_csv/retalhuleu.csv",
    "sacatepequez": "datos_csv/sacatepequez.csv",
    "sanmarcos": "datos_csv/sanmarcos.csv",
    "santarosa": "datos_csv/santarosa.csv",
    "solola": "datos_csv/solola.csv",
    "suchitipequez": "datos_csv/suchitipequez.csv",
    "totonicapan": "datos_csv/totonicapan.csv",
    "zacapa": "datos_csv/zacapa.csv",
}

# Carpeta de salida para los CSV limpios
out_dir = Path("salidas_limpias")
out_dir.mkdir(exist_ok=True, parents=True)
print("OK: entorno listo. Carpeta de salida:", out_dir.resolve())


OK: entorno listo. Carpeta de salida: C:\Users\Silvia\Documents\Cuarto Año\Segundo Semestre\Data\Proyecto1-DataScience-LimpiezaDeDatos\salidas_limpias


In [6]:
def load_and_fix_header(csv_path: str) -> pd.DataFrame:
    """
    Corrige el caso:
      - Fila 0: '0,1,2,...' (basura)
      - Fila 1: encabezado real ('CODIGO', 'DISTRITO', ...)
    Si no viene así, usa la primera fila como header.
    """
    raw = pd.read_csv(csv_path, header=None, dtype=str, keep_default_na=False)
    row0 = [str(x).strip() for x in raw.iloc[0].tolist()]
    row1 = [str(x).strip() for x in raw.iloc[1].tolist()] if len(raw) > 1 else None

    looks_like_numeric_header = all(re.fullmatch(r"\d+", x) for x in row0 if x != "")
    has_codigo_in_row1 = row1 is not None and any(x.upper() == "CODIGO" for x in row1)

    if looks_like_numeric_header and has_codigo_in_row1:
        header = row1
        df = raw.iloc[2:].copy()
    else:
        header = row0
        df = raw.iloc[1:].copy()

    df.columns = [str(c).strip() for c in header]
    for c in df.columns:
        df[c] = df[c].astype(str).str.strip()
    return df.reset_index(drop=True)

def standardize_colnames(cols):
    mapping = {
        "CODIGO":"CODIGO","DISTRITO":"DISTRITO","DEPARTAMENTO":"DEPARTAMENTO",
        "MUNICIPIO":"MUNICIPIO","ESTABLECIMIENTO":"ESTABLECIMIENTO","DIRECCION":"DIRECCION",
        "TELEFONO":"TELEFONO","SUPERVISOR":"SUPERVISOR","DIRECTOR":"DIRECTOR","NIVEL":"NIVEL",
        "SECTOR":"SECTOR","AREA":"AREA","STATUS":"STATUS","MODALIDAD":"MODALIDAD",
        "JORNADA":"JORNADA","PLAN":"PLAN","DEPARTAMENTAL":"DEPARTAMENTAL",
    }
    out = []
    for c in cols:
        key = str(c).strip().upper()
        out.append(mapping.get(key, key))
    return out


In [47]:
import pandas as pd

# Cargar el archivo unificado
df = pd.read_csv(r"salidas_limpias\establecimientos_union_limpio.csv")

# Validar columna CODIGO 
print("Duplicados por CODIGO:", df["CODIGO"].duplicated().sum())
print("Valores nulos en CODIGO:", df["CODIGO"].isna().sum())

# Limpiar espacios y asegurar que sea string
df["CODIGO"] = df["CODIGO"].astype(str).str.strip()

# Validar columnas de texto
cols_texto = ["DEPARTAMENTO", "MUNICIPIO", "SECTOR", "AREA"]

for col in cols_texto:
    print(f"\nColumna: {col}")
    print("Valores únicos antes de limpieza:", df[col].nunique())
    print("Ejemplos:", df[col].unique()[:10])  # primeras 10 categorías

    # Limpiar espacios, poner en título, quitar dobles espacios
    df[col] = df[col].astype(str).str.strip().str.title().str.replace(r'\s+', ' ', regex=True)

# 3. Validar TELEFONO 
print("\nValores únicos TELEFONO (antes de limpieza):", df["TELEFONO"].nunique())
df["TELEFONO"] = df["TELEFONO"].astype(str).str.replace(r'\D+', '', regex=True)  # dejar solo dígitos

#Longitudes de teléfono
df["long_tel"] = df["TELEFONO"].apply(len)
print("Distribución longitudes de teléfono:")
print(df["long_tel"].value_counts())

# Guardar la versión validada preliminar
df.drop(columns="long_tel").to_csv("salidas_limpias/salidas_validado.csv", index=False)
print("\nGuardado: salidas_limpias/salidas_validado.csv")


Duplicados por CODIGO: 6304
Valores nulos en CODIGO: 0

Columna: DEPARTAMENTO
Valores únicos antes de limpieza: 22
Ejemplos: ['ALTA VERAPAZ' 'BAJA VERAPAZ' 'CHIMALTENANGO' 'CHIQUIMULA'
 'CIUDAD CAPITAL' 'EL PROGRESO' 'ESCUINTLA' 'GUATEMALA' 'IZABAL' 'JALAPA']

Columna: MUNICIPIO
Valores únicos antes de limpieza: 315
Ejemplos: ['COBAN' 'SANTA CRUZ VERAPAZ' 'SAN CRISTOBAL VERAPAZ' 'TACTIC' 'TAMAHU'
 'SAN MIGUEL TUCURU' 'PANZOS' 'SENAHU' 'SAN PEDRO CARCHA'
 'SAN JUAN CHAMELCO']

Columna: SECTOR
Valores únicos antes de limpieza: 4
Ejemplos: ['PRIVADO' 'OFICIAL' 'MUNICIPAL' 'COOPERATIVA']

Columna: AREA
Valores únicos antes de limpieza: 3
Ejemplos: ['URBANA' 'RURAL' 'SIN ESPECIFICAR']

Valores únicos TELEFONO (antes de limpieza): 4013
Distribución longitudes de teléfono:
long_tel
8     12398
0        92
16       70
7        34
6         6
24        2
4         2
2         2
15        2
Name: count, dtype: int64

Guardado: salidas_limpias/salidas_validado.csv


In [7]:

for name, path in files.items():
    df = load_and_fix_header(path)
    df.columns = standardize_colnames(df.columns)

    # Resumen básico
    n_filas, n_cols = df.shape
    nulos = {c: int((df[c] == '').sum()) for c in df.columns}
    duplicados_codigo = int(df['CODIGO'].duplicated().sum()) if 'CODIGO' in df.columns else "N/A"

    # Teléfonos
    if 'TELEFONO' in df.columns:
        tel_digits = df['TELEFONO'].astype(str).str.replace(r"\D", "", regex=True)
        tel_longitudes = tel_digits.str.len().value_counts().sort_index().to_dict()
    else:
        tel_longitudes = {}

    print(f"--- {name.upper()} ---")
    print(f"Forma: {n_filas} filas x {n_cols} columnas")
    print(f"Duplicados por CODIGO: {duplicados_codigo}")
    print(f"Vacíos por columna: {nulos}")
    print(f"Longitudes en TELEFONO: {tel_longitudes}")
    print()


--- ALTAVERAPAZ ---
Forma: 295 filas x 17 columnas
Duplicados por CODIGO: 0
Vacíos por columna: {'CODIGO': 1, 'DISTRITO': 1, 'DEPARTAMENTO': 1, 'MUNICIPIO': 1, 'ESTABLECIMIENTO': 1, 'DIRECCION': 1, 'TELEFONO': 7, 'SUPERVISOR': 1, 'DIRECTOR': 1, 'NIVEL': 1, 'SECTOR': 1, 'AREA': 1, 'STATUS': 1, 'MODALIDAD': 1, 'JORNADA': 1, 'PLAN': 1, 'DEPARTAMENTAL': 1}
Longitudes en TELEFONO: {0: 7, 7: 1, 8: 286, 16: 1}

--- BAJAVERAPAZ ---
Forma: 95 filas x 17 columnas
Duplicados por CODIGO: 0
Vacíos por columna: {'CODIGO': 1, 'DISTRITO': 1, 'DEPARTAMENTO': 1, 'MUNICIPIO': 1, 'ESTABLECIMIENTO': 1, 'DIRECCION': 1, 'TELEFONO': 1, 'SUPERVISOR': 1, 'DIRECTOR': 1, 'NIVEL': 1, 'SECTOR': 1, 'AREA': 1, 'STATUS': 1, 'MODALIDAD': 1, 'JORNADA': 1, 'PLAN': 1, 'DEPARTAMENTAL': 1}
Longitudes en TELEFONO: {0: 1, 8: 92, 16: 2}

--- CHIMALTENANGO ---
Forma: 305 filas x 17 columnas
Duplicados por CODIGO: 0
Vacíos por columna: {'CODIGO': 1, 'DISTRITO': 1, 'DEPARTAMENTO': 1, 'MUNICIPIO': 1, 'ESTABLECIMIENTO': 1, 'DIRECCI

In [8]:
import pandas as pd
import re

# Limpiar y extraer solo dígitos del campo TELEFONO
tel_digits = df['TELEFONO'].fillna("").astype(str).str.replace(r"\D", "", regex=True)

# Teléfonos válidos deben tener exactamente 8 dígitos
# Y empezar con dígitos válidos para Guatemala: 2, 3, 4, 5, 6 o 7
valid_start_digits = ("2", "3", "4", "5", "6", "7")

# Crear máscara de teléfonos inválidos
invalid_mask = (
    (tel_digits.str.len() != 8) |                       # longitud distinta de 8
    (~tel_digits.str.startswith(valid_start_digits))    # o no empieza con dígito válido
)

# Filtrar los registros con problemas
df_invalid_tel = df.loc[invalid_mask, ["CODIGO", "ESTABLECIMIENTO", "DIRECCION", "TELEFONO"]]

# Mostrar resultados
print("Teléfonos inválidos encontrados:", len(df_invalid_tel))
display(df_invalid_tel.head(10))  # muestra los primeros 10 registros inválidos


# Textos con comillas dobles o espacios duplicados en ESTABLECIMIENTO y DIRECCION
weird_name_mask = df["ESTABLECIMIENTO"].str.contains(r"[\"'”“]{1}|  ", regex=True, na=False)
weird_dir_mask = df["DIRECCION"].str.contains(r"[\"'”“]{1}|  ", regex=True, na=False)

print("Establecimientos con comillas/espacios dobles:", weird_name_mask.sum())
display(df.loc[weird_name_mask, ["CODIGO", "ESTABLECIMIENTO"]].head(10))

print("Direcciones con comillas/espacios dobles:", weird_dir_mask.sum())
display(df.loc[weird_dir_mask, ["CODIGO", "DIRECCION"]].head(10))

# Posibles duplicados (mismo municipio + establecimiento + dirección, normalizados)
def normalize_text_basic(s: str) -> str:
    if pd.isna(s):
        return ""
    out = str(s).strip().upper()
    out = re.sub(r"[“”\"']", "", out)
    out = re.sub(r"\s+", " ", out)
    return out

def key_for_dupes(row):
    est = normalize_text_basic(row.get("ESTABLECIMIENTO", ""))
    dire = normalize_text_basic(row.get("DIRECCION", ""))
    muni = normalize_text_basic(row.get("MUNICIPIO", ""))
    return f"{muni} | {est} | {dire}"

keys = df.apply(key_for_dupes, axis=1)
counts = keys.value_counts()
dupe_keys = counts[counts > 1].head(10)

df_dupes = df.loc[keys.isin(dupe_keys.index), ["CODIGO", "MUNICIPIO", "ESTABLECIMIENTO", "DIRECCION", "JORNADA"]]
print("Posibles duplicados encontrados:", len(df_dupes))
display(df_dupes.head(20))


Teléfonos inválidos encontrados: 1


,CODIGO,ESTABLECIMIENTO,DIRECCION,TELEFONO
70,,,,


Establecimientos con comillas/espacios dobles: 23


,CODIGO,ESTABLECIMIENTO
0,19-01-0078-46,INSTITUTO DIVERSIFICADO ADSCRITO AL INSTITUTO ...
3,19-01-0086-46,"CENTRO EDUCATIVO JUVENIL CATÓLICO ""NUESTRA SEÑ..."
6,19-01-0160-46,"COLEGIO PARTICULAR MIXTO ""LICEO CRISTIANO ZACA..."
10,19-01-0192-46,"COLEGIO ""HIGA"""
11,19-01-0194-46,"COLEGIO ""HIGA"""
16,19-01-0209-46,"CENTRO DE EDUCACIÓN A DISTANCIA ""ITZAMNÁ"""
17,19-01-0211-46,"COLEGIO ""HIGA"""
26,19-01-0942-46,"CENTRO DE EDUCACIÓN A DISTANCIA ""ITZAMNÁ"""
28,19-01-1012-46,"CENTRO EDUCATIVO JUVENIL CATÓLICO ""NUESTRA SEÑ..."
37,19-04-0052-46,"COLEGIO PRIVADO MIXTO ""BETHANIA"""


Direcciones con comillas/espacios dobles: 2


,CODIGO,DIRECCION
0,19-01-0078-46,"7A. AVENIDA ""A"" FINAL Y 9A. CALLE ZONA 2, BARR..."
4,19-01-0116-46,"7A. AVENIDA ""A"" FINAL Y 9A. CALLE ZONA 2 BARRI..."


Posibles duplicados encontrados: 16


,CODIGO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,JORNADA
3,19-01-0086-46,ZACAPA,"CENTRO EDUCATIVO JUVENIL CATÓLICO ""NUESTRA SEÑ...",12 CALLE A 14-03 ZONA 1,MATUTINA
8,19-01-0175-46,ZACAPA,INSTITUTO TÉCNICO DE EDUCACIÓN INDUSTRIAL,"CIUDAD VICTORIA, BARRIO NUEVO",MATUTINA
10,19-01-0192-46,ZACAPA,"COLEGIO ""HIGA""",BOSQUES DE SAN JULIÁN ZACAPA,VESPERTINA
11,19-01-0194-46,ZACAPA,"COLEGIO ""HIGA""",BOSQUES DE SAN JULIÁN ZACAPA,DOBLE
18,19-01-0213-46,ZACAPA,INSTITUTO TÉCNICO PRIVADO VOCACIONAL,9A. CALLE 17-27 ZONA 3 BARRIO EL TAMARINDAL,SIN JORNADA
21,19-01-0304-46,ZACAPA,INSTITUTO TÉCNICO PRIVADO VOCACIONAL,9A. CALLE 17-27 ZONA 3 BARRIO EL TAMARINDAL,VESPERTINA
22,19-01-0332-46,ZACAPA,INSTITUTO TÉCNICO DE EDUCACIÓN INDUSTRIAL,"CIUDAD VICTORIA, BARRIO NUEVO",DOBLE
28,19-01-1012-46,ZACAPA,"CENTRO EDUCATIVO JUVENIL CATÓLICO ""NUESTRA SEÑ...",12 CALLE A 14-03 ZONA 1,MATUTINA
40,19-04-0071-46,GUALAN,"""LICEO SINAÍ""",BARRIO LAS FLORES,SIN JORNADA
41,19-04-0358-46,GUALAN,"""LICEO SINAÍ""",BARRIO LAS FLORES,VESPERTINA


In [18]:
import pandas as pd
import re

# Cargar el archivo (corrige la ruta según tu sistema operativo)
df = pd.read_csv("salidas_limpias/establecimientos_union_limpio.csv")

# Eliminar NaNs y extraer solo los dígitos numéricos
tel_digits = df['TELEFONO'].fillna("").astype(str).str.replace(r"\D", "", regex=True)

# Definir los dígitos válidos con los que puede comenzar un teléfono en Guatemala
valid_start_digits = ("2", "3", "4", "5", "6", "7")

# Teléfonos inválidos si:
# - No tienen exactamente 8 dígitos
# - O no empiezan con un dígito válido
invalid_mask = (
    (tel_digits.str.len() != 8) |
    (~tel_digits.str.startswith(valid_start_digits))
)

# Mostrar resultados
df_invalid_tel = df.loc[invalid_mask, ["CODIGO", "DEPARTAMENTO", "MUNICIPIO", "ESTABLECIMIENTO", "TELEFONO"]]

print("Teléfonos inválidos encontrados:", len(df_invalid_tel))
display(df_invalid_tel.head(10))


Teléfonos inválidos encontrados: 214


,CODIGO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,TELEFONO
83,16-03-0036-46,ALTA VERAPAZ,SAN CRISTOBAL VERAPAZ,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,NaN
110,16-03-1388-46,ALTA VERAPAZ,SAN CRISTOBAL VERAPAZ,COLEGIO MESON,79504027-79504028
136,16-06-2398-46,ALTA VERAPAZ,SAN MIGUEL TUCURU,INSTITUTO PRIVADO MIXTO DE MAGISTERIO BILINGUE...,NaN
151,16-07-0253-46,ALTA VERAPAZ,PANZOS,COLEGIO PRIVADO MIXTO JUAN AMOS COMENIO,4085613
170,16-09-0007-46,ALTA VERAPAZ,SAN PEDRO CARCHA,CENTRO EDUCATIVO NUEVA VIDA,NaN
207,16-09-9259-46,ALTA VERAPAZ,SAN PEDRO CARCHA,COLEGIO STHELLA DE HERNANDEZ,NaN
250,16-14-0054-46,ALTA VERAPAZ,CHAHAL,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,NaN
255,16-15-0002-46,ALTA VERAPAZ,FRAY BARTOLOME DE LAS CASAS,COLEGIO PARTICULAR MIXTO EL EXITO,NaN
300,15-01-0111-46,BAJA VERAPAZ,SALAMA,COLEGIO PARTICULAR MIXTO CIENCIA Y DESARROLLO,79540830-79540909
301,15-01-0112-46,BAJA VERAPAZ,SALAMA,COLEGIO PARTICULAR MIXTO CIENCIA Y DESARROLLO,79540830-79540909


In [46]:
# Limpieza masiva de todos los CSV

for name, path in files.items():
    # 1) Cargar y estandarizar encabezados
    dfi = load_and_fix_header(path)
    dfi.columns = standardize_colnames(dfi.columns)
    
    # Normalizar textos
    text_cols = [
        "DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION",
        "SUPERVISOR","DIRECTOR","NIVEL","SECTOR","AREA",
        "STATUS","MODALIDAD","JORNADA","PLAN","DEPARTAMENTAL"
    ]
    dfi = normalize_text_columns(dfi, text_cols)
    
    # Normalizar teléfonos
    dfi = normalize_phones_column(dfi, "TELEFONO")
    
    # Eliminar filas vacías o sin CODIGO
    mask_all_empty = dfi.replace("", np.nan).isna().all(axis=1)
    mask_codigo_empty = dfi["CODIGO"].astype(str).str.strip().eq("") if "CODIGO" in dfi.columns else False
    dfi = dfi.loc[~(mask_all_empty | mask_codigo_empty)].reset_index(drop=True)
    
    # Guardar CSV limpio
    out_path = out_dir / f"{name}_limpio.csv"
    dfi.to_csv(out_path, index=False, encoding="utf-8")
    print(f"Guardado: {out_path}")


Guardado: salidas_limpias\altaverapaz_limpio.csv
Guardado: salidas_limpias\bajaverapaz_limpio.csv
Guardado: salidas_limpias\chimaltenango_limpio.csv
Guardado: salidas_limpias\chiquimula_limpio.csv
Guardado: salidas_limpias\ciudadcapital_limpio.csv
Guardado: salidas_limpias\elprogreso_limpio.csv
Guardado: salidas_limpias\escuintla_limpio.csv
Guardado: salidas_limpias\guatemala_limpio.csv
Guardado: salidas_limpias\izabal_limpio.csv
Guardado: salidas_limpias\jalapa_limpio.csv
Guardado: salidas_limpias\jutiapa_limpio.csv
Guardado: salidas_limpias\peten_limpio.csv
Guardado: salidas_limpias\quetzaltenango_limpio.csv
Guardado: salidas_limpias\quiche_limpio.csv
Guardado: salidas_limpias\retalhuleu_limpio.csv
Guardado: salidas_limpias\sacatepequez_limpio.csv
Guardado: salidas_limpias\sanmarcos_limpio.csv
Guardado: salidas_limpias\santarosa_limpio.csv
Guardado: salidas_limpias\solola_limpio.csv
Guardado: salidas_limpias\suchitipequez_limpio.csv
Guardado: salidas_limpias\totonicapan_limpio.csv
Gu

In [19]:
import pandas as pd
import re

# Cargar el archivo CSV
df = pd.read_csv("salidas_limpias/establecimientos_union_limpio.csv")

# Limpiar y extraer solo los dígitos numéricos del teléfono
tel_digits = df['TELEFONO'].fillna("").astype(str).str.replace(r"\D", "", regex=True)

# Guardar longitud de cada teléfono en una columna auxiliar
df["TEL_DIGITS"] = tel_digits
df["TEL_LENGTH"] = tel_digits.str.len()

# Definir dígitos válidos para Guatemala
valid_start_digits = ("2", "3", "4", "5", "6", "7")

# Crear máscara de teléfonos inválidos
invalid_mask = (df["TEL_LENGTH"] != 8) | (~df["TEL_DIGITS"].str.startswith(valid_start_digits))

# Filtrar solo los inválidos
df_invalid = df.loc[invalid_mask, ["CODIGO", "DEPARTAMENTO", "MUNICIPIO", "ESTABLECIMIENTO", "TELEFONO", "TEL_DIGITS", "TEL_LENGTH"]]

# Mostrar resumen de longitudes de números inválidos
print("Resumen de longitudes en teléfonos inválidos:")
print(df_invalid["TEL_LENGTH"].value_counts().sort_index())

# Mostrar los primeros ejemplos
display(df_invalid.head(10))


Resumen de longitudes en teléfonos inválidos:
TEL_LENGTH
0     92
2      2
4      2
6      6
7     34
8      4
15     2
16    70
24     2
Name: count, dtype: int64


,CODIGO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,TELEFONO,TEL_DIGITS,TEL_LENGTH
83,16-03-0036-46,ALTA VERAPAZ,SAN CRISTOBAL VERAPAZ,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,NaN,,0
110,16-03-1388-46,ALTA VERAPAZ,SAN CRISTOBAL VERAPAZ,COLEGIO MESON,79504027-79504028,7950402779504028,16
136,16-06-2398-46,ALTA VERAPAZ,SAN MIGUEL TUCURU,INSTITUTO PRIVADO MIXTO DE MAGISTERIO BILINGUE...,NaN,,0
151,16-07-0253-46,ALTA VERAPAZ,PANZOS,COLEGIO PRIVADO MIXTO JUAN AMOS COMENIO,4085613,4085613,7
170,16-09-0007-46,ALTA VERAPAZ,SAN PEDRO CARCHA,CENTRO EDUCATIVO NUEVA VIDA,NaN,,0
207,16-09-9259-46,ALTA VERAPAZ,SAN PEDRO CARCHA,COLEGIO STHELLA DE HERNANDEZ,NaN,,0
250,16-14-0054-46,ALTA VERAPAZ,CHAHAL,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,NaN,,0
255,16-15-0002-46,ALTA VERAPAZ,FRAY BARTOLOME DE LAS CASAS,COLEGIO PARTICULAR MIXTO EL EXITO,NaN,,0
300,15-01-0111-46,BAJA VERAPAZ,SALAMA,COLEGIO PARTICULAR MIXTO CIENCIA Y DESARROLLO,79540830-79540909,7954083079540909,16
301,15-01-0112-46,BAJA VERAPAZ,SALAMA,COLEGIO PARTICULAR MIXTO CIENCIA Y DESARROLLO,79540830-79540909,7954083079540909,16


In [20]:
df_invalid[df_invalid["TEL_LENGTH"] == 8][["TELEFONO", "TEL_DIGITS"]]


,TELEFONO,TEL_DIGITS
6075,13012018,13012018
6775,00000000,00000000
10195,13012018,13012018
10895,00000000,00000000


In [21]:
df_invalid[df_invalid["TEL_LENGTH"] == 7][["TELEFONO", "TEL_DIGITS"]]


,TELEFONO,TEL_DIGITS
151,4085613,4085613
576,4225675,4225675
938,2232068,2232068
960,2223228,2223228
982,2232379,2232379
1295,5899624,5899624
1732,4215928,4215928
2335,4085613,4085613
2760,4225675,4225675
3122,2232068,2232068


In [22]:
# Agrupar todos los inválidos que NO tienen 7 ni 8 dígitos
otros_invalidos = df_invalid[~df_invalid["TEL_LENGTH"].isin([7, 8])]

# Mostrar resumen por longitud
print("Resumen por longitud:")
print(otros_invalidos["TEL_LENGTH"].value_counts().sort_index())

# Mostrar ejemplos por cada longitud
for length in sorted(otros_invalidos["TEL_LENGTH"].unique()):
    print(f"\n📏 Ejemplos de teléfonos con longitud {length}:")
    display(
        otros_invalidos[otros_invalidos["TEL_LENGTH"] == length][
            ["CODIGO", "DEPARTAMENTO", "TELEFONO", "TEL_DIGITS"]
        ].head(5)
    )


Resumen por longitud:
TEL_LENGTH
0     92
2      2
4      2
6      6
15     2
16    70
24     2
Name: count, dtype: int64

📏 Ejemplos de teléfonos con longitud 0:


,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
83,16-03-0036-46,ALTA VERAPAZ,NaN,
136,16-06-2398-46,ALTA VERAPAZ,NaN,
170,16-09-0007-46,ALTA VERAPAZ,NaN,
207,16-09-9259-46,ALTA VERAPAZ,NaN,
250,16-14-0054-46,ALTA VERAPAZ,NaN,



📏 Ejemplos de teléfonos con longitud 2:


,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
5361,01-17-0289-46,GUATEMALA,40,40
9481,01-17-0289-46,GUATEMALA,40,40



📏 Ejemplos de teléfonos con longitud 4:


...

,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
8375,08-01-0233-46,TOTONICAPAN,56769964-7766328,567699647766328
12495,08-01-0233-46,TOTONICAPAN,56769964-7766328,567699647766328



📏 Ejemplos de teléfonos con longitud 16:


,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
110,16-03-1388-46,ALTA VERAPAZ,79504027-79504028,7950402779504028
300,15-01-0111-46,BAJA VERAPAZ,79540830-79540909,7954083079540909
301,15-01-0112-46,BAJA VERAPAZ,79540830-79540909,7954083079540909
408,04-01-0139-46,CHIMALTENANGO,79649696-78739432,7964969678739432
409,04-01-0141-46,CHIMALTENANGO,78739432-79649696,7873943279649696



📏 Ejemplos de teléfonos con longitud 24:


,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
514,04-01-2994-46,CHIMALTENANGO,78392709-78396701-78396702,783927097839670178396702
2698,04-01-2994-46,CHIMALTENANGO,78392709-78396701-78396702,783927097839670178396702


In [23]:
def causa_probable(tel, length):
    if length == 0:
        return "Vacío"
    elif tel.startswith("502") and length in [11, 12, 15, 16]:
        return "Incluye código de país 502"
    elif "502" in tel and length == 24:
        return "Número duplicado con 502"
    elif length < 7:
        return "Número incompleto"
    elif length > 8:
        return "Número largo o concatenado"
    else:
        return "Otro"

print("📊 Clasificación de teléfonos inválidos (excepto 7 y 8 dígitos):\n")

for length in sorted(otros_invalidos["TEL_LENGTH"].unique()):
    subset = otros_invalidos[otros_invalidos["TEL_LENGTH"] == length]
    ejemplos = subset[["CODIGO", "DEPARTAMENTO", "TELEFONO", "TEL_DIGITS"]].head(5)

    # Detectar causa probable usando el primer ejemplo
    causas = [
        causa_probable(str(t), length) for t in subset["TEL_DIGITS"].head(5)
    ]

    print(f"📏 Longitud {length} → {len(subset)} registros")
    print("Causas probables en ejemplos:", set(causas))
    display(ejemplos)


📊 Clasificación de teléfonos inválidos (excepto 7 y 8 dígitos):

📏 Longitud 0 → 92 registros
Causas probables en ejemplos: {'Vacío'}


,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
83,16-03-0036-46,ALTA VERAPAZ,NaN,
136,16-06-2398-46,ALTA VERAPAZ,NaN,
170,16-09-0007-46,ALTA VERAPAZ,NaN,
207,16-09-9259-46,ALTA VERAPAZ,NaN,
250,16-14-0054-46,ALTA VERAPAZ,NaN,


📏 Longitud 2 → 2 registros
Causas probables en ejemplos: {'Número incompleto'}


,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
5361,01-17-0289-46,GUATEMALA,40,40
9481,01-17-0289-46,GUATEMALA,40,40


📏 Longitud 4 → 2 registros
Causas probables en ejemplos: {'Número incompleto'}


...

,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
8375,08-01-0233-46,TOTONICAPAN,56769964-7766328,567699647766328
12495,08-01-0233-46,TOTONICAPAN,56769964-7766328,567699647766328


📏 Longitud 16 → 70 registros
Causas probables en ejemplos: {'Número largo o concatenado'}


,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
110,16-03-1388-46,ALTA VERAPAZ,79504027-79504028,7950402779504028
300,15-01-0111-46,BAJA VERAPAZ,79540830-79540909,7954083079540909
301,15-01-0112-46,BAJA VERAPAZ,79540830-79540909,7954083079540909
408,04-01-0139-46,CHIMALTENANGO,79649696-78739432,7964969678739432
409,04-01-0141-46,CHIMALTENANGO,78739432-79649696,7873943279649696


📏 Longitud 24 → 2 registros
Causas probables en ejemplos: {'Número largo o concatenado'}


,CODIGO,DEPARTAMENTO,TELEFONO,TEL_DIGITS
514,04-01-2994-46,CHIMALTENANGO,78392709-78396701-78396702,783927097839670178396702
2698,04-01-2994-46,CHIMALTENANGO,78392709-78396701-78396702,783927097839670178396702


In [24]:
# Buscar instituciones con teléfonos de longitud 16 (dos números concatenados)
ejemplo_dos_numeros = df[df["TEL_DIGITS"].str.len() == 16]

# Mostrar la primera institución con dos números
print("Ejemplo de institución con dos números concatenados:")
display(ejemplo_dos_numeros[["CODIGO", "DEPARTAMENTO", "MUNICIPIO", "ESTABLECIMIENTO", "TELEFONO"]].head(1))


Ejemplo de institución con dos números concatenados:


,CODIGO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,TELEFONO
110,16-03-1388-46,ALTA VERAPAZ,SAN CRISTOBAL VERAPAZ,COLEGIO MESON,79504027-79504028


In [25]:
import re

def normalizar_lista_tel(s):
    partes = re.split(r"[-–—;,/| ]+", str(s))
    partes = [re.sub(r"\D", "", p) for p in partes if p]
    valid_start = tuple(list("234567"))
    partes = [p for p in partes if len(p)==8 and p.startswith(valid_start)]
    # quitar duplicados preservando orden
    seen = set()
    ordenadas = []
    for p in partes:
        if p not in seen:
            seen.add(p)
            ordenadas.append(p)
    return "; ".join(ordenadas) if ordenadas else ""

df_norm = df.copy()
df_norm["TELEFONO"] = df_norm["TELEFONO"].apply(normalizar_lista_tel)

print("Ejemplo normalizado en una celda:")
display(df_norm.loc[df_norm["CODIGO"]=="16-03-1388-46", ["CODIGO","TELEFONO"]].head(1))


Ejemplo normalizado en una celda:


,CODIGO,TELEFONO
110,16-03-1388-46,79504027; 79504028


In [26]:
def corregir_7digitos(num):
    if num.startswith(("22","23")):
        return "2" + num
    elif num.startswith(("40","41","42")):
        return "7" + num
    elif num.startswith(("24","31")):
        return "7" + num
    elif num.startswith(("55","53","58")):
        return "5" + num
    elif num.startswith(("78",)):
        return "7" + num
    else:
        return "X" + num  # si no entra en ningún patrón conocido

df["TELEFONO_CORREGIDO"] = df["TEL_DIGITS"].apply(lambda x: corregir_7digitos(str(x)) if len(str(x))==7 else x)


In [27]:
import re
import pandas as pd

df = df.copy()

# 1) Separar múltiples teléfonos por separadores comunes y "explotar" a filas
sep_pat = r"[-–—;,/| ]+"
tmp = df.assign(_part=df["TELEFONO"].astype(str).str.split(sep_pat)).explode("_part")

# 2) Limpiar cada parte dejando solo dígitos
tmp["_digits"] = tmp["_part"].fillna("").str.replace(r"\D", "", regex=True)

# 3) Función para corregir los de 7 dígitos (reglas que ya usamos)
def corregir_7(num7: str) -> str:
    if num7.startswith(("22","23")):   # fijos Ciudad de Guatemala
        return "2" + num7
    elif num7.startswith(("40","41","42","24","31")):  # fijos departamentos
        return "7" + num7
    elif num7.startswith(("55","53","58")):  # móviles antiguos
        return "5" + num7
    elif num7.startswith(("78",)):  # móviles/servicios según bloque
        return "7" + num7
    else:
        return ""  # no se corrige automáticamente (queda para revisión)

# 4) Aplicar corrección a 7 dígitos y conservar el resto tal cual
def normalize_digits(d):
    if not d:
        return ""
    if len(d) == 7:
        return corregir_7(d)
    return d

tmp["_norm"] = tmp["_digits"].apply(normalize_digits)

# 5) Validación final: teléfono válido = 8 dígitos y prefijo permitido
valid_start = tuple(list("234567"))
ok_mask = (tmp["_norm"].str.len() == 8) & (tmp["_norm"].str.startswith(valid_start))

df_valid = tmp.loc[ok_mask].drop(columns=["_part","_digits"]).rename(columns={"_norm":"TELEFONO"})
df_valid = df_valid.drop_duplicates(subset=["CODIGO","TELEFONO"])  # opcional

# 6) Lo pendiente por revisar (todo lo que no quedó válido)
df_pend = tmp.loc[~ok_mask, ["CODIGO","DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","TELEFONO","_digits","_norm"]]

print("✔ Teléfonos válidos (8 dígitos con prefijo correcto):", len(df_valid))
print("⚠ Pendientes por revisar manualmente:", len(df_pend))

# Resumen de pendientes por causa
def causa(digits, norm):
    if digits == "" or len(digits) == 0:
        return "Vacío"
    L = len(digits)
    if L in (2,4,6):
        return f"Fragmento ({L} díg)"
    if L == 7 and norm == "":
        return "7 díg sin regla clara"
    if L == 8 and not (norm.startswith(valid_start) if norm else False):
        return "8 díg prefijo inválido"
    if L == 15:
        return "Concatenado (8+7)"
    if L == 16:
        return "Concatenado (8+8)"
    if L == 24:
        return "Concatenado (8+8+8)"
    return f"Otro ({L} díg)"

df_pend["CAUSA"] = [causa(d, n) for d, n in zip(df_pend["_digits"], df_pend["_norm"])]

print("\nResumen pendientes por causa:")
print(df_pend["CAUSA"].value_counts().sort_index())

# Ejemplos rápidos
# display(df_valid.head(10))
# display(df_pend.head(10))


✔ Teléfonos válidos (8 dígitos con prefijo correcto): 6279
⚠ Pendientes por revisar manualmente: 114

Resumen pendientes por causa:
CAUSA
7 díg sin regla clara      2
8 díg prefijo inválido     4
Concatenado (8+8)          6
Fragmento (2 díg)          2
Fragmento (4 díg)          2
Fragmento (6 díg)          6
Vacío                     92
Name: count, dtype: int64


In [28]:
# Exportar los teléfonos pendientes a un CSV para revisión manual
df_pend.to_csv("telefonos_pendientes.csv", index=False, encoding="utf-8-sig")

print("Archivo generado: telefonos_pendientes.csv")
print("Registros pendientes:", len(df_pend))


Archivo generado: telefonos_pendientes.csv
Registros pendientes: 114


In [1]:
import re
import pandas as pd

# 1) Cargar datasets
df = pd.read_csv("establecimientos_validado_final.csv")
pendientes = pd.read_csv("telefonos_pendientes.csv")

# 2) Normalizar claves
if "CODIGO" not in df.columns or "CODIGO" not in pendientes.columns:
    raise ValueError("Falta la columna 'CODIGO' en alguno de los archivos.")

# 3) Unificar: usar teléfono de pendientes cuando exista
# (Asegura que en df solo quede 1 columna de teléfono antes del merge)
df = df.drop(columns=[c for c in df.columns if c != "TELEFONO" and "TELEFONO" in c], errors="ignore")

df = df.merge(
    pendientes[["CODIGO", "TELEFONO"]],
    on="CODIGO",
    how="left",
    suffixes=("", "_CORR")
)

# Si hay teléfono corregido en pendientes, usarlo
df["TELEFONO"] = df["TELEFONO_CORR"].combine_first(df["TELEFONO"])
df = df.drop(columns=["TELEFONO_CORR"])

# 4) Normalizar TELEFONO (quitar .0, separar, dejar solo válidos de 8 dígitos que empiezan 2-7)
valid_start = set("234567")

sep_pat = r"[-–—;/,| ]+"  # separadores más comunes

def normalize_tel(val):
    if pd.isna(val):
        return ""
    s = str(val).strip()
    # quitar artefacto de float
    s = re.sub(r"\.0$", "", s)
    # dividir por separadores
    parts = re.split(sep_pat, s)
    cleaned = []
    for p in parts:
        d = re.sub(r"\D", "", p)  # dejar solo dígitos
        if len(d) == 8 and (d[0] in valid_start):
            cleaned.append(d)
    # quitar duplicados preservando orden
    out = []
    seen = set()
    for d in cleaned:
        if d not in seen:
            seen.add(d)
            out.append(d)
    return "; ".join(out)  # si hay varios, se quedan en la misma celda

df["TELEFONO"] = df["TELEFONO"].apply(normalize_tel)

# 5) (Opcional) eliminar cualquier otra columna auxiliar relacionada a teléfono
aux_patterns = [
    r"^TELEFONO_", r"^TEL_", r"^_digits$", r"^_norm$", r"^TELDIGITS$", r"^TELLENGTH$"
]
drop_mask = df.columns.str.contains("|".join(aux_patterns), case=False, regex=True) & (df.columns != "TELEFONO")
df = df.drop(columns=df.columns[drop_mask], errors="ignore")

# 6) Validación: filas que quedaron SIN ningún teléfono válido (vacías)
sin_tel_mask = (df["TELEFONO"].astype(str).str.len() == 0)
print("📌 Filas sin teléfono válido después de normalizar:", sin_tel_mask.sum())

# Normalizar y reemplazar "Ciudad Capital" -> "Guatemala" en DEPARTAMENTO
df["DEPARTAMENTO"] = df["DEPARTAMENTO"].astype(str)

# Quita acentos y homogeneiza para comparar
dep_norm = (
    df["DEPARTAMENTO"]
      .str.normalize("NFKD").str.encode("ascii","ignore").str.decode("ascii")
      .str.strip().str.upper().str.replace(r"\s+", " ", regex=True)
)

mask_ciudad_capital = dep_norm.isin([
    "CIUDAD CAPITAL", "CIUDADCAPITAL"
])

reemplazados = int(mask_ciudad_capital.sum())
df.loc[mask_ciudad_capital, "DEPARTAMENTO"] = "Guatemala"

print(f"🔁 Reemplazados 'Ciudad Capital' -> 'Guatemala': {reemplazados}")

import unicodedata

# Función para quitar tildes y pasar a mayúsculas
def normalizar_texto(s):
    if pd.isna(s):  # Evita errores con NaN
        return s
    s = str(s).upper().strip()
    # Quitar tildes
    s = ''.join(c for c in unicodedata.normalize('NFD', s) 
                if unicodedata.category(c) != 'Mn')
    # Quitar espacios dobles
    s = ' '.join(s.split())
    return s

# Aplicar a todas las columnas de tipo texto
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].apply(normalizar_texto)

# Guardar CSV limpio
df.to_csv("establecimientos_limpios.csv", index=False)
print("✅ Archivo limpio generado: establecimientos_limpios.csv")


📌 Filas sin teléfono válido después de normalizar: 46
🔁 Reemplazados 'Ciudad Capital' -> 'Guatemala': 866
✅ Archivo limpio generado: establecimientos_limpios.csv


In [45]:
# Revisar longitudes de los teléfonos finales
df["TEL_LENGTH"] = df["TELEFONO"].astype(str).str.len()

# Filtrar los que NO son de 8 dígitos
telefonos_invalidos = df[df["TEL_LENGTH"] != 8]

print("📌 Teléfonos con longitud distinta a 8 dígitos:", len(telefonos_invalidos))
print(telefonos_invalidos["TEL_LENGTH"].value_counts())

# Mostrar ejemplos
print("\nEjemplos de teléfonos no válidos:")
print(telefonos_invalidos[["CODIGO", "DEPARTAMENTO", "MUNICIPIO", "ESTABLECIMIENTO", "TELEFONO"]].head(20))


📌 Teléfonos con longitud distinta a 8 dígitos: 76
TEL_LENGTH
0     46
18    29
28     1
Name: count, dtype: int64

Ejemplos de teléfonos no válidos:
            CODIGO   DEPARTAMENTO              MUNICIPIO  \
111  16-03-1388-46   Alta Verapaz  San Cristobal Verapaz   
153  16-07-0253-46   Alta Verapaz                 Panzos   
254  16-14-0054-46   Alta Verapaz                 Chahal   
255  16-14-0054-46   Alta Verapaz                 Chahal   
306  15-01-0111-46   Baja Verapaz                 Salama   
307  15-01-0112-46   Baja Verapaz                 Salama   
415  04-01-0139-46  Chimaltenango          Chimaltenango   
416  04-01-0141-46  Chimaltenango          Chimaltenango   
417  04-01-0142-46  Chimaltenango          Chimaltenango   
475  04-01-0347-46  Chimaltenango          Chimaltenango   
521  04-01-2994-46  Chimaltenango          Chimaltenango   
583  04-04-0074-46  Chimaltenango      San Juan Comalapa   
709  20-01-0041-46     Chiquimula             Chiquimula   
710  20-01-